# Caching Embeddings

Embeddings can be stored or temporarily cached to avoid needing to recompute them.

Caching embeddings can be done using a `CacheBackedEmbeddings`.

The cache backed embedder is a wrapper around an embedder that caches
embeddings in a key-value store. 

The text is hashed and the hash is used as the key in the cache.


The main supported way to initialized a `CacheBackedEmbeddings` is `from_bytes_store`. This takes in the following parameters:

- underlying_embedder: The embedder to use for embedding.
- document_embedding_cache: The cache to use for storing document embeddings.
- namespace: (optional, defaults to `""`) The namespace to use for document cache. This namespace is used to avoid collisions with other caches. For example, set it to the name of the embedding model used.

**Attention**: Be sure to set the `namespace` parameter to avoid collisions of the same text embedded using different embeddings models.

In [3]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryStore
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings

## In Memory

This section shows how to set up an in memory cache for embeddings. This type of cache is primarily 
useful for unit tests or prototyping. Do **not** use this cache if you need to actually store the embeddings.

In [4]:
store = InMemoryStore()

In [5]:
underlying_embeddings = OpenAIEmbeddings()
embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

In [6]:
%%time
embeddings = embedder.embed_documents(["hello", "goodbye"])

CPU times: user 390 ms, sys: 47.8 ms, total: 438 ms
Wall time: 857 ms


The second time we try to embed the embedding time is only 2 ms because the embeddings are looked up in the cache.

In [7]:
%%time
embeddings_from_cache = embedder.embed_documents(["hello", "goodbye"])

CPU times: user 1.5 ms, sys: 576 µs, total: 2.08 ms
Wall time: 2.09 ms


In [8]:
embeddings == embeddings_from_cache

True

## File system

This section covers how to use a file system store.

In [9]:
fs = LocalFileStore("./cache/")

In [10]:
embedder2 = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, fs, namespace=underlying_embeddings.model
)

In [11]:
%%time
embeddings = embedder2.embed_documents(["hello", "goodbye"])

CPU times: user 3.51 ms, sys: 0 ns, total: 3.51 ms
Wall time: 3.09 ms


In [12]:
%%time
embeddings = embedder2.embed_documents(["hello", "goodbye"])

CPU times: user 2.26 ms, sys: 708 µs, total: 2.97 ms
Wall time: 2.5 ms


Here are the embeddings that have been persisted to the directory `./cache`. 

Notice that the embedder takes a namespace parameter.

In [13]:
list(fs.yield_keys())

['text-embedding-ada-002614d7cf6-46f1-52fa-9d3a-740c39e7a20e',
 'text-embedding-ada-0020fc1ede2-407a-5e14-8f8f-5642214263f5',
 'text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 'text-embedding-ada-002e4ad20ef-dfaa-5916-9459-f90c6d8e8159',
 'text-embedding-ada-002a5ef11e4-0474-5725-8d80-81c91943b37f',
 'text-embedding-ada-00281426526-23fe-58be-9e84-6c7c72c8ca9a',
 'text-embedding-ada-00217a6727d-8916-54eb-b196-ec9c9d6ca472',
 'text-embedding-ada-002305efb5c-3f01-5657-bcf2-2b92fb1747ca',
 'text-embedding-ada-002704c76af-3696-5383-9858-6585616669ef',
 'text-embedding-ada-0029c15af4c-0e3b-5020-926f-752f448be568',
 'text-embedding-ada-00201dbc21f-5e4c-5fb5-8d13-517dbe7a32d4',
 'text-embedding-ada-0028061c36f-1bb7-5be5-9de2-afbed11ff5f2',
 'text-embedding-ada-0025990258b-0781-5651-8444-c69cb5b6da3d',
 'text-embedding-ada-002b793db35-a909-5ba0-8c51-314dc776017d',
 'text-embedding-ada-002b35583f7-c036-5cae-b60b-394754b7d149',
 'text-embedding-ada-0021297d37a-2bc1-5e19-bf13-6c950f0

## Redis Store



In [19]:
from langchain.storage import RedisStore

In [28]:
# For cache isolation can use a separate DB
# Or additional namepace
store = RedisStore(redis_url="redis://localhost:6379", client_kwargs={'db': 2}, namespace='embedding_caches')

underlying_embeddings = OpenAIEmbeddings()
embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

In [29]:
%%time
embeddings = embedder.embed_documents(["hello", "goodbye"])

CPU times: user 5.71 ms, sys: 6.53 ms, total: 12.2 ms
Wall time: 599 ms


In [30]:
%%time
embeddings = embedder.embed_documents(["hello", "goodbye"])

CPU times: user 3.36 ms, sys: 0 ns, total: 3.36 ms
Wall time: 2.99 ms


In [31]:
list(store.yield_keys())

['text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 'text-embedding-ada-0026ba52e44-59c9-5cc9-a084-284061b13c80']

In [34]:
list(store.client.scan_iter())

[b'embedding_caches/text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 b'text-embedding-ada-0026ba52e44-59c9-5cc9-a084-284061b13c80',
 b'embedding_caches/text-embedding-ada-0026ba52e44-59c9-5cc9-a084-284061b13c80',
 b'text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5']

## Using with a vectorstore

Let's see this cache in action with the FAISS vectorstore.

In [37]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [38]:
fs = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    OpenAIEmbeddings(), fs, namespace=underlying_embeddings.model
)

The cache is empty prior to embedding

In [39]:
list(fs.yield_keys())

['text-embedding-ada-002614d7cf6-46f1-52fa-9d3a-740c39e7a20e',
 'text-embedding-ada-0020fc1ede2-407a-5e14-8f8f-5642214263f5',
 'text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 'text-embedding-ada-002e4ad20ef-dfaa-5916-9459-f90c6d8e8159',
 'text-embedding-ada-002a5ef11e4-0474-5725-8d80-81c91943b37f',
 'text-embedding-ada-00281426526-23fe-58be-9e84-6c7c72c8ca9a',
 'text-embedding-ada-00217a6727d-8916-54eb-b196-ec9c9d6ca472',
 'text-embedding-ada-002305efb5c-3f01-5657-bcf2-2b92fb1747ca',
 'text-embedding-ada-002704c76af-3696-5383-9858-6585616669ef',
 'text-embedding-ada-0029c15af4c-0e3b-5020-926f-752f448be568',
 'text-embedding-ada-00201dbc21f-5e4c-5fb5-8d13-517dbe7a32d4',
 'text-embedding-ada-0028061c36f-1bb7-5be5-9de2-afbed11ff5f2',
 'text-embedding-ada-0025990258b-0781-5651-8444-c69cb5b6da3d',
 'text-embedding-ada-002b793db35-a909-5ba0-8c51-314dc776017d',
 'text-embedding-ada-002b35583f7-c036-5cae-b60b-394754b7d149',
 'text-embedding-ada-0021297d37a-2bc1-5e19-bf13-6c950f0

Load the document, split it into chunks, embed each chunk and load it into the vector store.

In [30]:
raw_documents = TextLoader("../state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

create the vectorstore

In [31]:
%%time
db = FAISS.from_documents(documents, cached_embedder)

CPU times: user 33.3 ms, sys: 275 µs, total: 33.6 ms
Wall time: 33 ms


If we try to create the vectostore again, it'll be much faster since it does not need to re-compute any embeddings.

In [32]:
%%time
db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 28.5 ms, sys: 4.11 ms, total: 32.6 ms
Wall time: 31.7 ms


And here are some of the embeddings that got created:

In [33]:
list(fs.yield_keys())[:5]

['text-embedding-ada-002614d7cf6-46f1-52fa-9d3a-740c39e7a20e',
 'text-embedding-ada-0020fc1ede2-407a-5e14-8f8f-5642214263f5',
 'text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 'text-embedding-ada-002e4ad20ef-dfaa-5916-9459-f90c6d8e8159',
 'text-embedding-ada-002a5ef11e4-0474-5725-8d80-81c91943b37f']